In [70]:
os.environ["OPENAI_API_KEY"] = <OPENAI-API-KEY>
os.environ["HUGGINGFACEHUB_API_TOKEN"] = <HUGGINGFACEHUB-API-TOKEN>

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


### Using OpenAI 

In [71]:
from langchain.llms import OpenAI
from langchain import HuggingFaceHub
import os

In [72]:
llm = OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6)

In [69]:
text = "Name the largest mammal?"
print(llm.predict(text))



The largest mammal is the blue whale.


### Using a HuggingFace Model

In [73]:
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0, "max_length":64})

In [74]:
output = llm_huggingface.predict("Give me a short poem on AI")
print(output)

i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love


In [75]:
output2= llm.predict("Give me a short poem on AI")
print(output2)



Artificial intelligence, a marvel of technology
Created by humans, with endless possibility
From robots to chatbots, it's a world of AI
Where machines can learn, think and even try

With algorithms and data, it's constantly evolving
Making our lives easier, problem-solving
But some fear its power, and what it may bring
As we give machines the ability to think

Yet AI has brought us so far
Helping us reach for the stars
But let us not forget, it's just a tool
Humanity's creativity will always rule

So let's embrace this wonder, with caution and care
For in the hands of humans, AI can be rare
A tool for progress, a tool for good
Let's use it wisely, as we should.


### Prompt templates

In [76]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables = ['country'], template="Tell me the capital of {country}")
  
prompt_template.format(country="Croatia")

'Tell me the capital of Croatia'

In [77]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)
print(chain.run("Croatia"))



The capital of Croatia is Zagreb.


### Combining Multiple Chains Using simple Sequential Chain

In [78]:
prompt_template_0=PromptTemplate(input_variables = ['country'], template="Tell me the capital of {country}") 
capital_chain = LLMChain(llm=llm, prompt=prompt_template_0,output_key="capital")

In [79]:
prompt_template_1=PromptTemplate(input_variables = ['capital'], template="Tell me the food places to visit the capital mentioned {capital}") 
famous_chain=LLMChain(llm=llm, prompt= prompt_template_1,output_key="places")

In [80]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[capital_chain,famous_chain], input_variables=['country'],output_variables=['capital','places'])
chain({'country':"Russia"}) 

{'country': 'Russia',
 'capital': '\n\nThe capital of Russia is Moscow.',
 'places': ' Some popular food places to visit in Moscow are:\n\n1. Café Pushkin - This elegant restaurant serves traditional Russian cuisine in a luxurious setting.\n\n2. White Rabbit - Located on the 16th floor, this restaurant offers stunning views of the city along with modern Russian cuisine.\n\n3. Stolovaya 57 - This Soviet-style cafeteria offers a variety of affordable and authentic Russian dishes.\n\n4. Mari Vanna - This cozy restaurant serves homestyle Russian dishes, and has a charming ambiance.\n\n5. Berezka - This upscale restaurant specializes in Russian and European cuisine, and is known for its caviar and seafood dishes.\n\n6. Varenichnaya No. 1 - A popular chain of restaurants serving delicious and affordable Ukrainian and Russian dishes.\n\n7. Mu-Mu - Another chain of budget-friendly cafes serving traditional Russian food, including borscht and pelmeni.\n\n8. Glavpivtorg - This pub-style restaura

 ### ChatOpenAI

In [81]:
from langchain_openai import ChatOpenAI

In [82]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [83]:
chatllm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [84]:
chatllm([
    SystemMessage(content="You are a AI powered travel assistant"),
    HumanMessage(content="Please provide some solo traveling tips for travelling in USA")
])

AIMessage(content="Sure! Here are some solo traveling tips for traveling in the USA:\n\n1. Research your destination: Before you travel, research the area you will be visiting including the local customs, language, transportation options, and safety tips.\n\n2. Stay connected: Make sure to have a working phone with you at all times and keep important phone numbers handy in case of emergencies.\n\n3. Share your itinerary: Let someone know where you will be and when you expect to return. Check in with them regularly to keep them updated on your whereabouts.\n\n4. Pack light: Traveling solo means you are responsible for carrying your luggage, so pack light to make your journey easier.\n\n5. Stay alert: Pay attention to your surroundings and trust your instincts. Avoid risky areas, especially at night, and be cautious when interacting with strangers.\n\n6. Stay in safe accommodations: Choose reputable hotels or hostels with good reviews to ensure your safety and comfort during your stay.\n

### Prompt Template + LLM + Output Parsers

In [85]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [86]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [87]:
template = "You are a helpful assistant. When I give you any input, you generate 5 words related to input which would be comma separated"
human_template = "{text}"
chatprompt= ChatPromptTemplate.from_messages(
    [
        ("system",template),
        ("human",human_template)
    ]
)

In [88]:
chain = chatprompt | chatllm | Commaseperatedoutput()

In [89]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' bright', ' knowledgeable', ' astute']